# Overview
* Not concearn about the order
* Try to predict current economic phase (not forward)
* Normalized data 
* No Dimensionality  Reduncion
* Not transforming the variables
* Not lagging the variables


In [ ]:
import pandas as pd
import numpy as np
import missingno
import matplotlib.pyplot as plt 
import seaborn as sns
plt.style.use('ggplot')
from fredapi import Fred
from utils.key import APIKEY as API_FRED
import utils_FRED_MD.model_evaluation  as me
from utils_FRED_MD.format_datasets import  get_nonNANs_fullsample



In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score ,confusion_matrix ,precision_score, recall_score, f1_score, classification_report ,roc_curve, roc_auc_score ,roc_curve, auc, ConfusionMatrixDisplay , RocCurveDisplay


from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
import statsmodels.api as smf

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import utils_FRED_MD.model_evaluation  as me
SPLIT = 0.7
SEED=42
mms = MinMaxScaler()
scaler = StandardScaler()


In [ ]:
df = get_nonNANs_fullsample()
df.head()

In [ ]:
df.tail()

In [ ]:
df.isna().any().sum()

In [ ]:
df.shape

In [ ]:
sns.countplot(x= "USREC",data=df)
plt.show()

In [ ]:
df.set_index("date")['USREC'].plot()

In [ ]:

output_directory  ='img'
# Define the numeric columns
numeric_columns = df.drop(['date','USREC'],axis=1).columns

# Loop through numeric columns
for col in numeric_columns:
    # Create a new figure for each numeric variable
    plt.figure(figsize=(10, 5))
    
    # Loop through unique classes in the categorical column
    for category in df['USREC'].unique():
        # Filter the DataFrame by the current category
        category_data = df[df['USREC'] == category][col]
        
        # Plot a histogram for the current class
        sns.histplot(category_data, label=category, kde=True)  # You can use sns.histplot or sns.distplot
        
    # Set plot labels, title, and legend
    plt.xlabel(col)
    plt.title(f'Distribution of {col} by Economic cycle')
    plt.legend()
    
    # Show or save the plot
    plt.show()


    # Save the plot as an image in the "Img" folder
    # output_path = os.path.join(output_directory, f'{col}_distribution.pdf')
    # plt.savefig(output_path)
    
    # Show or clear the plot
    plt.close()
    


------------------

## Testing each model 



### Probit

In [ ]:

X_norm = mms.fit_transform(X)
X_scaled = scaler.fit_transform(X)

In [ ]:
probit_model=smf.Probit(y,X)
result=probit_model.fit()
print(result.summary2())

In [ ]:
probit_model=smf.Probit(y,X_norm)
result=probit_model.fit(maxiter=1000)
print(result.summary2())

In [ ]:
probit_model=smf.Probit(y,X_scaled)
result=probit_model.fit()
print(result.summary2())

In [ ]:
from sklearn import metrics



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT, random_state=SEED, stratify=y)


X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)


probit=smf.Probit(y,X)
probit.fit(maxiter=500)
print(probit.fit().summary2())

### Logistic  Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT, random_state=SEED, stratify=y)

log_reg = LogisticRegression(class_weight='balanced',solver='liblinear',random_state = SEED)




X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

log_reg.fit(X_train_norm, y_train)

train_metrics = me.get_metrics(log_reg,y_train,X_train_norm )
test_metrics = me.get_metrics(log_reg,y_test,X_test_norm )
me.display_metrics(train_metrics,test_metrics)

me.plot_classification_metrics(log_reg,y_train,X_train_norm )
me.plot_classification_metrics(log_reg,y_test,X_test_norm )



In [ ]:
feature_importance_log_reg = pd.DataFrame({"Varariables":X.columns,"Coefs":log_reg.coef_[0]})


In [ ]:
help(LogisticRegression)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT, random_state=SEED, stratify=y)



X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)



strat_kfold = StratifiedKFold(n_splits=10, shuffle=False)

params_log_reg = {
    'C': [0.001, 0.01, 0.1, 1, 5,10,8,30,50, 100, 1000],
    'penalty': ['elasticnet', 'l1', 'l2'],
    "solver":['saga',"lblinear"]
}



# Step 2: Perform cross-validation with hyperparameter tuning

grid_search_log_reg = GridSearchCV(LogisticRegression(class_weight='balanced',random_state = SEED), params_log_reg, cv=strat_kfold, scoring='roc_auc',n_jobs=-1)
grid_search_log_reg.fit(X_train_norm, y_train)


best_log_reg = grid_search_log_reg.best_estimator_

best_log_reg.fit(X_train_norm, y_train)

train_metrics = me.get_metrics(best_log_reg,y_train,X_train_norm )
test_metrics = me.get_metrics(best_log_reg,y_test,X_test_norm )
me.display_metrics(train_metrics,test_metrics)

me.plot_classification_metrics(best_log_reg,y_train,X_train_norm )
me.plot_classification_metrics(best_log_reg,y_test,X_test_norm )



In [ ]:
grid_search_log_reg.best_estimator_

In [ ]:
grid_search_log_reg.best_params_['penalty']

In [ ]:
grid_search_log_reg.best_params_

In [ ]:
grid_search_log_reg. best_score_

### SVM Linear

In [ ]:
from sklearn.svm import SVC
import utils_FRED_MD.model_evaluation as me

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT, random_state=SEED, stratify=y)



X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)



strat_kfold = StratifiedKFold(n_splits=10, shuffle=False)

params_svm_linear = {
    'C': [0.001, 0.01, 0.1, 1, 5,10,8,30,50, 100, 1000]
}



# Step 2: Perform cross-validation with hyperparameter tuning

grid_search_svm_linear = GridSearchCV(SVC(kernel='linear',random_state = SEED), params_svm_linear, cv=strat_kfold, scoring='roc_auc',n_jobs=-1)
grid_search_svm_linear.fit(X_train_norm, y_train)


best_svm_linear = grid_search_svm_linear.best_estimator_

best_svm_linear.fit(X_train_norm, y_train)

train_metrics = me.get_metrics(best_svm_linear,y_train,X_train_norm )
test_metrics = me.get_metrics(best_svm_linear,y_test,X_test_norm )
me.display_metrics(train_metrics,test_metrics)

me.plot_classification_metrics(best_svm_linear,y_train,X_train_norm )
me.plot_classification_metrics(best_svm_linear,y_test,X_test_norm )



In [ ]:
grid_search_svm_linear.best_params_

### SVM Kernel trick

In [ ]:
from sklearn.svm import SVC
import utils_FRED_MD.model_evaluation as me

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT, random_state=SEED, stratify=y)



X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)



strat_kfold = StratifiedKFold(n_splits=10, shuffle=False)

params_svm_kernel = {
    'C': [0.001, 0.01, 0.1, 1, 5,10,8,30,50, 100, 1000]
}



# Step 2: Perform cross-validation with hyperparameter tuning

grid_search_svm_kernel = GridSearchCV(SVC(kernel='rbf',random_state = SEED), params_svm_kernel, cv=strat_kfold, scoring='roc_auc',n_jobs=-1)
grid_search_svm_kernel.fit(X_train_norm, y_train)


best_svm_kernel = grid_search_svm_kernel.best_estimator_

best_svm_kernel.fit(X_train_norm, y_train)

train_metrics = me.get_metrics(best_svm_kernel,y_train,X_train_norm )
test_metrics = me.get_metrics(best_svm_kernel,y_test,X_test_norm )
me.display_metrics(train_metrics,test_metrics)

me.plot_classification_metrics(best_svm_kernel,y_train,X_train_norm )
me.plot_classification_metrics(best_svm_kernel,y_test,X_test_norm )



In [ ]:
grid_search_svm_kernel.best_params_

### Decision trees

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT, random_state=SEED, stratify=y)



X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

strat_kfold = StratifiedKFold(n_splits=10, shuffle=False)

params_decision_tree = {
    'criterion': ['gini', 'entropy','log_loss'],
    'max_depth': [None, 10,30, 60, 90],
    'min_samples_split': [2, 5, 10, 20, 50],
    'min_samples_leaf': [1, 2, 4, 8, 16],
    'max_features': ['auto', 'sqrt', 'log2', None]
}




# Step 2: Perform cross-validation with hyperparameter tuning

grid_search_DT = GridSearchCV(DecisionTreeClassifier(random_state = SEED), params_decision_tree, cv=strat_kfold, scoring='roc_auc',n_jobs=-1)
grid_search_DT.fit(X_train_norm, y_train)


best_DT = grid_search_DT.best_estimator_

best_DT.fit(X_train_norm, y_train)

train_metrics = me.get_metrics(best_DT,y_train,X_train_norm )
test_metrics = me.get_metrics(best_DT,y_test,X_test_norm )
me.display_metrics(train_metrics,test_metrics)

me.plot_classification_metrics(best_DT,y_train,X_train_norm )
me.plot_classification_metrics(best_DT,y_test,X_test_norm )



### Random Forests

In [ ]:
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT, random_state=SEED, stratify=y)



# X_train_norm = mms.fit_transform(X_train)
# X_test_norm = mms.transform(X_test)

# strat_kfold = StratifiedKFold(n_splits=10, shuffle=False)

# params_random_forest = {
#     'n_estimators': [10, 50, 100, 200],
#     'criterion': ['gini', 'entropy'],
#     'max_depth': [None, 10, 20, 30, 40, 50],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }





# # Step 2: Perform cross-validation with hyperparameter tuning

# grid_search_RF = GridSearchCV(RandomForestClassifier(random_state=SEED), params_decision_tree, cv=strat_kfold, scoring='roc_auc',n_jobs=-1)
# grid_search_RF.fit(X_train_norm, y_train)


# best_RF = grid_search_RF.best_estimator_

# best_RF.fit(X_train_norm, y_train)

# train_metrics = me.get_metrics(best_RF,y_train,X_train_norm )
# test_metrics = me.get_metrics(best_RF,y_test,X_test_norm )
# me.display_metrics(train_metrics,test_metrics)

# me.plot_classification_metrics(best_RF,y_train,X_train_norm )
# me.plot_classification_metrics(best_RF,y_test,X_test_norm )



-----


##  Consolidating the experiments

In [ ]:
X = df.drop(['USREC','date'],axis=1)
y = df['USREC']

In [ ]:


%load_ext autoreload
%autoreload 2

from utils_FRED_MD.ml_execution import evaluate_models

results = evaluate_models(X,y,scoring='roc_auc')



In [ ]:

train_results = pd.concat([ i['Metrics_Train'] for i in results] ).sort_values('Area Under the Curve',ascending=False)
train_results

In [ ]:
results[0].keys()

In [ ]:

test_results = pd.concat([ i['Metrics_Test'] for i in results] ).sort_values('Area Under the Curve',ascending=False)
test_results

In [19]:
df['USREC'].shift(-1)

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
771    0.0
772    0.0
773    0.0
774    0.0
775    NaN
Name: USREC, Length: 776, dtype: float64

In [ ]:
list(range(0,12))

In [21]:
import os 
i=0
os.makedirs(os.path.dirname('img/' + f"Predictions_{i}_ahead"), exist_ok=True)

In [22]:
os.path.dirname('img/' + f"Predictions_{i}_ahead")

'img'

In [24]:
df

,date,RPI,W875RX1,DPCERA3M086SBEA,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,INVEST,USREC
0,1959-01-01,2583.560,2426.0,15.188,18235.77392,21.9665,23.3891,22.2688,31.7011,19.0149,...,29.600,15.164,63.517,18.294,10.152,2.13,2.45,2.04,84.2043,0.0
1,1959-02-01,2593.596,2434.8,15.346,18369.56308,22.3966,23.7048,22.4617,31.9337,19.1147,...,29.600,15.179,63.554,18.302,10.167,2.14,2.46,2.05,83.5280,0.0
2,1959-03-01,2610.396,2452.7,15.491,18523.05762,22.7193,23.8483,22.5719,31.9337,19.4890,...,29.600,15.189,63.634,18.289,10.185,2.15,2.45,2.07,81.6405,0.0
3,1959-04-01,2627.446,2470.0,15.435,18534.46600,23.2032,24.1927,22.9026,32.4374,19.6138,...,29.600,15.219,63.698,18.300,10.221,2.16,2.47,2.08,81.8099,0.0
4,1959-05-01,2642.720,2486.4,15.622,18679.66354,23.5528,24.3936,23.1231,32.5925,20.0130,...,29.600,15.227,63.804,18.280,10.238,2.17,2.48,2.08,80.7315,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,2023-04-01,19030.119,15600.2,115.246,684636.00000,103.1274,101.7771,102.4474,103.6524,108.9423,...,291.138,119.893,108.514,118.685,122.165,29.23,33.90,26.04,5243.7520,0.0
772,2023-05-01,19074.384,15636.1,115.400,689158.00000,102.7864,101.3461,101.8586,102.8448,109.5420,...,291.510,120.020,108.772,118.279,122.441,29.33,33.97,26.15,5210.2067,0.0
773,2023-06-01,19082.740,15654.8,115.693,690518.00000,102.3673,100.6659,100.9632,101.5389,106.0073,...,292.077,120.221,108.390,118.369,122.797,29.52,34.04,26.40,5188.5697,0.0
774,2023-07-01,19087.488,15684.9,116.432,693668.00000,103.0920,101.4893,102.0583,102.7410,108.9151,...,292.663,120.478,107.615,118.353,123.352,29.63,34.23,26.49,5166.1852,0.0


In [20]:

%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

def ml_pipeline(original_dataframe,shift_target=0,**kwargs):
    modeling_dataframe = original_dataframe.copy()
    modeling_dataframe['Target'] = modeling_dataframe['USREC'].shift(-shift_target)
    modeling_dataframe.dropna(inplace=True)
    X = modeling_dataframe.drop(['USREC','date','Target'],axis=1)
    y = modeling_dataframe['Target']
    results = evaluate_models(X,y,scoring='roc_auc',**kwargs)


    train_stats = pd.concat([ i['Metrics_Train'] for i in results] ).sort_values('Area Under the Curve',ascending=False)
    
    test_stats = pd.concat([ i['Metrics_Test'] for i in results] ).sort_values('Area Under the Curve',ascending=False)

    train_test_stats_df = pd.concat([train_metrics,test_stats])

    train_test_stats_df['Prediction_ahead'] = shift_target
    return results , train_test_stats_df



all_results = []
list_table_results = []

for i in [0,1,3,6,12]:
    roc_save_path = 'img/' + f"Predictions_{i}_ahead"
    iteration_results = ml_pipeline(df,shift_targe=i,save_path=roc_save_path)
    list_results =  iteration_results[0]
    all_results.append(list_results)

    table_results = iteration_results[1]
    list_table_results.append(table_results)




0
1
2
3
4
5
6
7
8
9
10
11
